In [1]:
import gym
import collections
import numpy as np
from ppo_torch import *
from utils import plotLearning
import pickle as pkl
from tqdm import tqdm

### Main

In [7]:
env = gym.make('LunarLander-v2')
env.observation_space.shape

(8,)

In [2]:
if __name__ == '__main__':
    env_name = 'CartPole-v0'
    env =gym.make(env_name)
    
    directory = 'D:\Reinforcement Learning\My codes\PPO/'
    
    if (not os.path.exists(directory+'models/'+str(env_name)+'/')):
        os.makedirs(directory+'models/'+str(env_name)+'/')
    if (not os.path.exists(directory + 'plots/'+str(env_name)+'/')):
        os.makedirs(directory+'plots/'+str(env_name)+'/')
    if (not os.path.exists(directory+'pickle/'+str(env_name)+'/')):
        os.makedirs(directory+'pickle/'+str(env_name)+'/')
    
    N=20
    batch_size = 5
    n_epochs = 4
    alpha = 0.0003
    
    n_games = 10
    
    figure_file = directory+'plots/'+str(env_name)+'/'+env_name +'.png'
    pickle_file  = directory+'pickle/'+str(env_name)+'/'
    model_file  =directory+'models/'+str(env_name)+'/'
    
    
    agent = Agent(n_actions= env.action_space.n, batch_size = batch_size,
                  alpha = alpha, n_epochs=n_epochs, 
                  input_dims= env.observation_space.shape, pickle_file=pickle_file, model_file=model_file)
    
    best_score = env.reward_range[0]
    score_history = []
    
    best_avg_score = -np.inf
    learn_iters = 0
    avg_score = 0
    n_steps = 0
    prev_n_games= 0 
    
    load_checkpoint = True
    train = True
    
    if load_checkpoint:
        agent.load_models()
        
        score_history = pkl.load(agent.pklObj)
        prev_n_games = pkl.load(agent.pklObj)
        n_steps = pkl.load(agent.pklObj)
        learn_iters = pkl.load(agent.pklObj)
    
    for i in range(n_games):
        observation = env.reset()
        done = False
        score = 0
        while not done:
            action, prob, val = agent.choose_action(observation)
            
            observation_, reward, done, info = env.step(action)
            score += reward
            
            n_steps+=1
            
            agent.remember(observation, action, prob, val, reward, done)
            if n_steps % N ==0:
                if train:
                    agent.learn()
                    learn_iters += 1
                
            observation = observation_
            
        score_history.append(score)
        
        avg_score = np.mean(score_history[-20:])
            
        if train or (train and (i == n_games-1)):    
            if avg_score > best_avg_score:
                best_avg_score = avg_score
                agent.save_models()
                
                pkl.dump(score_history, agent.pklObj)
                pkl.dump(i, agent.pklObj)
                pkl.dump(n_steps, agent.pklObj)
                pkl.dump(learn_iters, agent.pklObj)
            
                
        print('Episode ',i , 'Score %.1f' %score, 'Avg score %.1f' %avg_score,
                'time_steps', n_steps, 'learning_steps', learn_iters)
            
    x = [i+1 for i in range(len(score_history))]
    plotLearning(x= x,scores= score_history, filename= figure_file, window=20)

... loading models ...


EOFError: Ran out of input